<a href="https://colab.research.google.com/github/liangjieyu123/CropIntensity/blob/main/%E6%89%8B%E5%86%99%E6%95%B0%E5%AD%97%E8%AF%86%E5%88%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import Compose,ToTensor,Normalize

In [2]:
batch_size=128
def get_dataloader(train=True):
  transform_fn=Compose([
      ToTensor(),
      Normalize(mean=(0.1307,),std=(0.3081,))
  ])
  dataset = MNIST(root="./data",train=train,download=True,transform=transform_fn)
  data_loader = DataLoader(dataset,batch_size=batch_size,shuffle=True)
  return data_loader

In [3]:
class MnistNet(nn.Module):
  def __init__(self):
    super(MnistNet,self).__init__()
    self.fc1=nn.Linear(28*28*1,28)
    self.fc2=nn.Linear(28,10)

  def forward(self,x):
    x=x.view([-1,28*28*1])
    x=self.fc1(x)
    x=F.relu(x)
    x=self.fc2(x)
    out=F.log_softmax(x,dim=-1)
    return out

In [4]:
if torch.cuda.is_available():
  print("Cuda is available")
else:
  print("Cuda is unavailable")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Cuda is unavailable


In [15]:
from torch.optim import Adam
mnist_net=MnistNet().to(device)
optimizer = Adam(mnist_net.parameters(),lr=0.001)
def train(epoch):
  mode=True
  mnist_net.train(mode=mode)

  train_dataloader = get_dataloader()
  for idx,(input,traget) in enumerate(train_dataloader):
    input = input.to(device)
    traget = traget.to(device)
    output = mnist_net(input)
    loss = F.nll_loss(output,traget)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if idx%100 ==0:
      print('Train Epoch:{}[{}/{}({:.0f}%)]\tLoss:{:.6f}'.format(
          epoch,idx*len(input),len(train_dataloader.dataset),
          100.*idx/len(train_dataloader),loss.item()
      ))
      # print(epoch,idx,loss.item())

In [20]:
# 模型测试
def test():
  test_loss=0
  correct=0
  mnist_net.eval()
  test_dataloader = get_dataloader(train=False)
  with torch.no_grad():
    for input,traget in test_dataloader:
      output = mnist_net(input)
      test_loss += F.nll_loss(output,traget,reduction='sum').item()
      pred = output.data.max(1,keepdim=True)[1]   #获取最大值的位置
      correct += pred.eq(traget.data.view_as(pred)).sum()
  test_loss /= len(test_dataloader.dataset)
  print('\nTest set:Avg. loss:{:.4f},Accracy:{}/{}({:.2f}%)\n'.format(
      test_loss,correct,len(test_dataloader.dataset),
      100.*correct/len(test_dataloader.dataset)
  )) 

test()



Test set:Avg. loss:0.1746,Accracy:9501/10000(95.01%)



In [16]:
if __name__=='__main__':
  test()
  for i in range(3):
    train(i)
    test()

Train Epoch:0[0/60000(0%)]	Loss:2.299897
Train Epoch:0[12800/60000(21%)]	Loss:0.341031
Train Epoch:0[25600/60000(43%)]	Loss:0.234435
Train Epoch:0[38400/60000(64%)]	Loss:0.218512
Train Epoch:0[51200/60000(85%)]	Loss:0.437395
Train Epoch:1[0/60000(0%)]	Loss:0.286224
Train Epoch:1[12800/60000(21%)]	Loss:0.376066
Train Epoch:1[25600/60000(43%)]	Loss:0.294707
Train Epoch:1[38400/60000(64%)]	Loss:0.131206
Train Epoch:1[51200/60000(85%)]	Loss:0.222621
Train Epoch:2[0/60000(0%)]	Loss:0.110209
Train Epoch:2[12800/60000(21%)]	Loss:0.274264
Train Epoch:2[25600/60000(43%)]	Loss:0.319275
Train Epoch:2[38400/60000(64%)]	Loss:0.131517
Train Epoch:2[51200/60000(85%)]	Loss:0.147178


In [ ]:
# 模型保存
torch.save(mnist_net.state_dict(),"save/mnist_net.pt")  #保存模型参数
torch.save(optimizer.state_dict(),"save/mnist_optimizer.pt")

In [ ]:
# 模型加载
mnist_net.load_state_dict(torch.load("data/mnist_net.pt"))
optimizer.load_state_dict(torch.load("data/mnist_optimizer.pt"))

In [ ]:
# 模型测试
def test():
  loss_list = []
  acc_list = []
  test_dataloader = get_dataloader(train=False)
  for idx,(input,traget) in enumerate(test_dataloader):
    with torch.no_grad():
      output = mnist_net(input)
      cur_loss = F.nll_loss(output,traget)
      loss_list.append(cur_loss)
      # 计算准确率
      pred = output.max(dim=-1)[-1]
      cur_acc = pred.eq(traget).float().mean()
      acc_list.append(cur_acc)
  print("平均精度：",np.mean(acc_list),"平均损失：",np.mean(loss_list)) 

test()


import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/model/pytorch-lightning-unet/train_img"
os.chdir(path)
os.listdir(path)
